In [12]:
import requests
import numpy as np
import pandas as pd

In [18]:
BASE = "https://api.elections.kalshi.com/trade-api/v2/markets"

params = {"limit": 1000, "status": "open", "mve_filter": "exclude"}  # max per page
all_markets = []
cursor = None

while True:
    if cursor:
        params["cursor"] = cursor
    resp = requests.get(BASE, params=params)
    resp.raise_for_status()
    data = resp.json()

    all_markets.extend(data["markets"])
    cursor = data.get("cursor")
    if not cursor:  # no more pages
        break
    print(f"Loaded {len(all_markets)} so far")
print(len(all_markets), "markets fetched")


Loaded 1000 so far
Loaded 2000 so far
Loaded 3000 so far
Loaded 4000 so far
Loaded 5000 so far
Loaded 6000 so far
Loaded 7000 so far
Loaded 8000 so far
Loaded 9000 so far
Loaded 10000 so far
Loaded 11000 so far
Loaded 12000 so far
Loaded 13000 so far
Loaded 14000 so far
Loaded 15000 so far
Loaded 16000 so far
Loaded 17000 so far
Loaded 18000 so far
Loaded 19000 so far
Loaded 20000 so far
20889 markets fetched


In [129]:
def market_is_multivariate(market):
    return market.get("mve_collection_ticker") not in (None, "", 0)

SPORT_PREFIXES = (
    "KXNFL",
    "KXNHL",
    "KXNBA",
    "KXNCAA",
    "KXNCBB",
    "KXCRICKET",
    "KXSERIEA",
    "KXLEADERNFL",
    "KXEPL",
    # add more as you discover them
)

def is_sports_market(m: dict) -> bool:
    event_ticker = m.get("event_ticker", "")
    is_sports = event_ticker.startswith(SPORT_PREFIXES)
    is_sports = is_sports or event_ticker.split('-')[0].endswith(('GAME', 'CUP', 'MATCH'))
    return is_sports


liquid_markets = [market for market in all_markets if market['volume_24h'] > -1]
print(f"Of {len(all_markets)} markets, {len(liquid_markets)} markets are liquid")
# pd.Series([market_is_multivariate(market) for market in all_markets]).value_counts()

Of 20889 markets, 20889 markets are liquid


In [130]:
display(pd.Series([is_sports_market(market) for market in liquid_markets]).value_counts())
event_tickers = [market['event_ticker'].split('-')[0] for market in liquid_markets]
print(f"Of {len(event_tickers)} events, there are {len(np.unique(event_tickers))} unique ones")

False    14241
True      6648
Name: count, dtype: int64

Of 20889 events, there are 1774 unique ones


In [131]:
market_df = pd.DataFrame(all_markets)
market_df['event_proxy'] = market_df.event_ticker.map(lambda x : x.split('-')[0])
market_df['is_sports_market'] = market_df.apply(is_sports_market, axis = 1)
market_df['volume_bucket'] = pd.cut(market_df.volume_24h, [-1, 0, 1, 10, 100, 1000, 10000, 1e9])
display_df = market_df.groupby('is_sports_market').volume_bucket.value_counts().unstack(0)
display_df.loc['total'] = display_df.sum(axis = 0)
display_df

is_sports_market,False,True
volume_bucket,,
"(-1.0, 0.0]",10097,4336
"(0.0, 1.0]",137,82
"(1.0, 10.0]",423,297
"(10.0, 100.0]",1183,641
"(100.0, 1000.0]",1531,734
"(1000.0, 10000.0]",683,421
"(10000.0, 1000000000.0]",187,137
total,14241,6648


In [132]:
market_df['close_time'] = pd.to_datetime(market_df.close_time)
market_df[(~market_df.is_sports_market) & (market_df.volume_24h > -1)].drop_duplicates('event_proxy', keep = 'last').close_time.describe(percentiles = np.arange(0, 1, 0.1))

count                                   1591
mean     2027-07-10 17:58:23.775612672+00:00
min                2026-01-02 07:45:00+00:00
0%                 2026-01-02 07:45:00+00:00
10%                2026-02-01 15:00:00+00:00
20%                2026-06-02 02:00:00+00:00
30%                2026-11-03 15:00:00+00:00
40%                2027-01-01 04:59:00+00:00
50%                2027-01-01 15:00:00+00:00
60%                2027-02-01 15:00:00+00:00
70%                2027-11-03 15:00:00+00:00
80%                2027-11-03 15:00:00+00:00
90%                2029-01-20 15:00:00+00:00
max                2099-08-01 04:59:00+00:00
Name: close_time, dtype: object

In [134]:
jan_markets = market_df[(~market_df.is_sports_market) & (market_df.volume_24h > -1) & (market_df.close_time < pd.Timestamp('2026-01-31').tz_localize("America/New_York"))].drop_duplicates('event_proxy', keep = 'last')
print(len(jan_markets))
for _, row in jan_markets.iterrows():
    print(row.title, "|", row.event_ticker)

132
BTC price up in next 15 mins? | KXBTC15M-26JAN020245
ETH price up in next 15 mins? | KXETH15M-26JAN020245
Top USA Song on Spotify on Jan 2, 2026? | KXSPOTIFYD-26JAN02
Top Global Song on Spotify on Jan 2, 2026? | KXSPOTIFYGLOBALD-26JAN02
Will the runner-up top Song on Jan 2, 2026 be 4 Raws? | KXSPOTIFY2D-26JAN02
Will the high temp in Chicago be 24-25° on Jan 2, 2026? | KXHIGHCHI-26JAN02
Will the **high temp in Miami** be 69-70° on Jan 2, 2026? | KXHIGHMIA-26JAN02
Will the **high temp in Austin** be 84-85° on Jan 2, 2026? | KXHIGHAUS-26JAN02
Will the **high temp in Philadelphia** be 33-34° on Jan 2, 2026? | KXHIGHPHIL-26JAN02
Will the **high temp in NYC** be 28-29° on Jan 2, 2026? | KXHIGHNY-26JAN02
Will the **high temp in Denver** be 57-58° on Jan 2, 2026? | KXHIGHDEN-26JAN02
Will the minimum temperature be  32-33° on Jan 2, 2026? | KXLOWTDEN-26JAN02
Will the minimum temperature be  46-47° on Jan 2, 2026? | KXLOWTMIA-26JAN02
Will the minimum temperature be  17-18° on Jan 2, 2026? | 

In [48]:
[market for market in liquid_markets if not is_sports_market(market)][100]

{'can_close_early': True,
 'category': '',
 'close_time': '2026-01-31T04:59:00Z',
 'created_time': '2025-12-31T18:29:15.899062Z',
 'event_ticker': 'KXAAAGASM-26JAN31',
 'expected_expiration_time': '2026-01-31T15:00:00Z',
 'expiration_time': '2026-02-07T15:00:00Z',
 'expiration_value': '',
 'floor_strike': 2.95,
 'last_price': 7,
 'last_price_dollars': '0.0700',
 'latest_expiration_time': '2026-02-07T15:00:00Z',
 'liquidity': 81045,
 'liquidity_dollars': '810.4500',
 'market_type': 'binary',
 'no_ask': 93,
 'no_ask_dollars': '0.9300',
 'no_bid': 91,
 'no_bid_dollars': '0.9100',
 'no_sub_title': 'Above 2.95',
 'notional_value': 100,
 'notional_value_dollars': '1.0000',
 'open_interest': 1364,
 'open_time': '2025-12-31T22:00:00Z',
 'previous_price': 22,
 'previous_price_dollars': '0.2200',
 'previous_yes_ask': 24,
 'previous_yes_ask_dollars': '0.2400',
 'previous_yes_bid': 17,
 'previous_yes_bid_dollars': '0.1700',
 'price_level_structure': 'linear_cent',
 'price_ranges': [{'end': '1.0000

In [128]:
x = [market for market in all_markets if market['event_ticker'].startswith('KXAAAGASM')]
print(len(x))

for xx in x:
    print(xx['title'])

10
Will average **gas prices** be above $2.95?
Will average **gas prices** be above $2.70?
Will average **gas prices** be above $2.75?
Will average **gas prices** be above $2.80?
Will average **gas prices** be above $2.85?
Will average **gas prices** be above $3.30?
Will average **gas prices** be above $3.20?
Will average **gas prices** be above $3.10?
Will average **gas prices** be above $3.00?
Will average **gas prices** be above $2.90?
